#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Monthly Budget Mover Parameters
Apply the previous month's budget/spend delta to the current month.  Aggregate up the budget and spend from the previous month of each category declared then apply the delta of the spend and budget equally to each Line Item under that Category.
 1. No changes made can be made in DV360 from the start to the end of this process
 1. Make sure there is budget information for the current and previous month's IOs in DV360
 1. Make sure the provided spend report has spend data for every IO in the previous month
 1. Spend report must contain 'Revenue (Adv Currency)' and 'Insertion Order ID'
 1. There are no duplicate IO Ids in the categories outlined below
 1. This process must be ran during the month of the budget it is updating
 1. If you receive a 502 error then you must separate your jobs into two, because there is too much information being pulled in the sdf
 1. Manually run this job
 1. Once the job has completed go to the table for the new sdf and export to a csv
 1. Take the new sdf and upload it into DV360
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'recipe_name': '',  # 
  'partner_id': '',  # The sdf file types.
  'auth_write': 'service',  # Credentials used for writing data.
  'auth_read': 'user',  # Credentials used for reading data.
  'budget_categories': '{}',  # A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}
  'filter_ids': [],  # Comma separated list of filter ids for the request.
  'excluded_ios': '',  # A comma separated list of Inserion Order Ids that should be exluded from the budget calculations
  'version': '5',  # The sdf version to be returned.
  'is_colab': True,  # Are you running this in Colab? (This will store the files in Colab instead of Bigquery)
  'dataset': '',  # Dataset that you would like your output tables to be produced in.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Monthly Budget Mover
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'auth': 'user',
      'description': 'Create a dataset where data will be combined and transfored for upload.',
      'dataset': {'field': {'kind': 'string','name': 'dataset','order': 1,'description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'timeout': 90,
        'body': {
          'metadata': {
            'dataRange': 'PREVIOUS_MONTH',
            'format': 'CSV',
            'title': {'field': {'prefix': 'Monthly_Budget_Mover_','description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'kind': 'string'}}
          },
          'params': {
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_INSERTION_ORDER'
            ],
            'metrics': [
              'METRIC_REVENUE_ADVERTISER'
            ]
          },
          'timezoneCode': {'field': {'description': 'Timezone for report dates.','kind': 'timezone','name': 'recipe_timezone','default': 'America/Los_Angeles'}}
        },
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'description': 'The comma separated list of Advertiser Ids.','name': 'filter_ids','order': 7,'default': '','kind': 'integer_list'}}
          }
        }
      },
      'delete': False
    }
  },
  {
    'monthly_budget_mover': {
      'out_old_sdf': {
        'bigquery': {
          'table': {'field': {'prefix': 'SDF_OLD_','description': '','name': 'recipe_name','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','order': 8,'default': '','kind': 'string'}},
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'skip_rows': 0
        },
        'file': '/content/old_sdf.csv'
      },
      'report_name': {'field': {'prefix': 'Monthly_Budget_Mover_','description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'kind': 'string'}},
      'is_colab': {'field': {'description': 'Are you running this in Colab? (This will store the files in Colab instead of Bigquery)','name': 'is_colab','order': 7,'default': True,'kind': 'boolean'}},
      'excluded_ios': {'field': {'kind': 'integer_list','name': 'excluded_ios','order': 4,'description': 'A comma separated list of Inserion Order Ids that should be exluded from the budget calculations'}},
      'auth': 'user',
      'out_changes': {
        'bigquery': {
          'table': {'field': {'prefix': 'SDF_BUDGET_MOVER_LOG_','description': '','name': 'recipe_name','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','order': 8,'default': '','kind': 'string'}},
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'skip_rows': 0
        },
        'file': '/content/log.csv'
      },
      'sdf': {
        'partner_id': {'field': {'kind': 'integer','name': 'partner_id','order': 1,'description': 'The sdf file types.'}},
        'create_single_day_table': False,
        'table_suffix': '',
        'version': {'field': {'kind': 'choice','order': 6,'choices': ['SDF_VERSION_5','SDF_VERSION_5_1'],'description': 'The sdf version to be returned.','default': '5','name': 'version'}},
        'read': {
          'filter_ids': {
            'single_cell': True,
            'values': {'field': {'description': 'Comma separated list of filter ids for the request.','name': 'filter_ids','order': 4,'default': [],'kind': 'integer_list'}}
          }
        },
        'dataset': {'field': {'description': 'Dataset to be written to in BigQuery.','name': 'dataset','order': 6,'default': '','kind': 'string'}},
        'auth': 'user',
        'filter_type': 'FILTER_TYPE_ADVERTISER_ID',
        'time_partitioned_table': False,
        'file_types': 'INSERTION_ORDER'
      },
      'out_new_sdf': {
        'bigquery': {
          'table': {'field': {'prefix': 'SDF_NEW_','description': '','name': 'recipe_name','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','order': 8,'default': '','kind': 'string'}},
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'skip_rows': 0
        },
        'file': '/content/new_sdf.csv'
      },
      'budget_categories': {'field': {'description': 'A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}','name': 'budget_categories','order': 3,'default': '{}','kind': 'json'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
